# Recreating ABM in Python

In [1]:
# imports
import numpy as np
import pandas as pd

In [2]:
# colors for each bacteria in the graphics
colors = {"coop": "blue", "cheat": "red"}

## Bacteria class (Agent)

In [ ]:
# bacteria class (agent) - CA
class Bacteria:
    def __init__(self, energy, recombination, breed, loc):
        # int for bacteria energy
        self.energy = energy
        # boolean for if it takes part in recombination
        self.recombination = recombination
        # int for breed
        self.breed = breed
        # list of ints for location [x, y]
        self.loc = loc
        # string for the color according to the breed
        self.color = colors[breed]
    
    # method to set color based on breed
    def set_breed(self, breed):
        self.breed = breed
        self.color = colors[breed]

    # method to get possible move options
    def get_move_options(self, size):
        options = []  # to store possible move options
        # check if x-coordinate is not at left edge of grid
        if self.loc[0] != 0:
            options.append([self.loc[0] - 1, self.loc[1]])
        # check if x-coordinate is not at right edge of grid
        if self.loc[0] != size[0] - 1:
            options.append([self.loc[0] + 1, self.loc[1]])
        # check if y-coordinate is not at top edge of grid
        if self.loc[1] != 0:
            options.append([self.loc[0], self.loc[1] - 1])
        # check if y-coordinate is not at bottom edge of grid
        if self.loc[1] != size[1] - 1:
            options.append([self.loc[0], self.loc[1] + 1])
        return options

## Simulation class (Model)

In [ ]:
# simulation class - we can remove print statements, only for testing - CA
# TODO: add tracking and return of simulation data over generations
# TODO: add animation of simulation based on tracking of data over generations

# simulation class
class Sim:
    def __init__(self, 
                 start_energy, 
                 population_size, 
                 population_viscosity, 
                 recombination_cost,
                 mutation_rate, 
                 recombination_rate, 
                 percent_recombination, 
                 percent_cooperation,
                 contribution, 
                 multiplier, 
                 max_x, 
                 max_y):
        
        # attribute initialization
        self.start_energy = start_energy  # energy is consumed over time. when energy is 0, bacteria dies
        self.population_size = population_size  # when cells die, remaining cells reproduce to keep population size constant
        self.population_viscosity = population_viscosity  # no of gens each cell remains in same loc before moving to adjacent loc
        self.recombination_cost = recombination_cost  # subtracts from resources available to each cell that recombines
        self.mutation_rate = mutation_rate  # rate at which coops produce cheat offspring
        self.recombination_rate = recombination_rate  # rate at which recombs change phenotype of other cells in same loc (coops make others coop, cheats make others cheat, converted cells are also recomb)
        self.percent_recombination = percent_recombination  # initial percent of recombiners
        self.percent_cooperation = percent_cooperation  # initial percent of cooperators
        self.contribution = contribution  # all coops contribute some fitness to common pool
        self.multiplier = multiplier  # common pool value is multiplied (benefit of cooperation)
        self.max_x = max_x  # x-coordinate of grid
        self.max_y = max_y  # y-coordinate of grid
        
        # list to store bacteria
        self.bacteria = []

        # number of recombination and cooperation bacteria
        self.num_rec = int(np.round(population_size * percent_recombination))  # total population size * percent recombiners, round to nearest int
        self.num_coop = int(np.round(population_size * percent_cooperation))  # total population size * percent cooperators, round to nearest int

        np.random.seed(0)

        # randomly assign recombination and cooperation to bacteria
        index_rec = np.random.choice(population_size, self.num_rec, replace=False)
        index_coop = np.random.choice(population_size, self.num_coop, replace=False)

        # create bacteria
        for i in np.arange(population_size):
            bac = Bacteria(start_energy, True if i in index_rec else False, "coop" if i in index_coop else "cheat",
                           [np.random.choice(max_x, 1)[0], np.random.choice(max_y,1)[0]])
            self.bacteria.append(bac)
    
    # method to simulate a generation
    def simulate_gen(self, gen):
        #print("starting gen", gen)
        # coops contribute public goods at the cost of energy
        def contribute(bac):
            if bac.breed == "coop":
                bac.energy -= self.contribution  # coops lose energy
            return bac
        # apply contribute function to all bacteria
        self.bacteria = list(map(contribute, self.bacteria))

        # all cells benefit from public good
        def benefit(bac, amt):
            bac.energy += amt
            return bac
        # apply benefit function to all bacteria
        self.bacteria = list(map(benefit, self.bacteria, [(self.num_coop * self.multiplier) / len(self.bacteria)]
                                 * len(self.bacteria)))
        
        # lose fitness
        def update_fitness(bac):
            if np.random.choice(2, 1) == 1:
                bac.energy -= 1
            if bac.recombination:
                bac.energy -= self.recombination_cost
            return bac
        self.bacteria = list(map(update_fitness, self.bacteria))

        # bacteria die if fitness is < 0
        alive_bac = []
        for b in self.bacteria:
            if b.energy < 0:
                if b.breed == "coop":
                    self.num_coop -= 1
                if b.recombination:
                    self.num_rec -= 1
            else:
                alive_bac.append(b)
        self.bacteria = alive_bac
        if len(self.bacteria) == 0:
            return

        # repopulation
        while len(self.bacteria) < self.population_size:
            parent = self.bacteria[np.random.choice(len(self.bacteria), 1)[0]]
            mutate = np.random.choice(2, 1, p=[1 - self.mutation_rate, self.mutation_rate])[0]
            if mutate == 1:
                breed = parent.breed
            elif parent.breed == "coop":
                breed = "cheat"
            else:
                breed = "coop"
            if breed == "coop":
                self.num_coop += 1
            if parent.recombination:
                self.num_rec += 1
            bac = Bacteria(parent.energy, parent.recombination, breed, parent.loc)
            self.bacteria.append(bac)

        # conversion/recombination
        if self.num_rec != 0:
            # bacterias in each location
            bac_locs = {}
            # number of recombination bacterias in each location
            bac_rec = {}
            for b in self.bacteria:
                if str(b.loc) in bac_locs.keys():
                    bac_locs[str(b.loc)].append(b)
                    if b.recombination:
                        bac_rec[str(b.loc)] += 1
                else:
                    bac_locs[str(b.loc)] = [b]
                    if b.recombination:
                        bac_rec[str(b.loc)] = 1
                    else:
                        bac_rec[str(b.loc)] = 0
            for k in bac_locs.keys():
                if bac_rec[k] == 0 or bac_rec[k] == len(bac_locs[k]):
                    continue
                for b in bac_locs[k]:
                    if not b.recombination:
                        # pick whether to recombine or not for eah recombination bacteria
                        recombine = np.random.choice(2, bac_rec[k], p=[1-self.recombination_rate, self.recombination_rate])
                        if np.sum(recombine) > 0:
                            b.recombination = True
                            self.num_rec += 1

        # movement
        if (gen + 1) % self.population_viscosity == 0:
            for b in self.bacteria:
                move_ops = b.get_move_options([self.max_x, self.max_y])
                b.loc = move_ops[np.random.choice(len(move_ops), 1)[0]]
        
    def simulate(self, gens):
        print("starting simulation")
        for g in np.arange(gens):
            self.simulate_gen(g)
            if len(self.bacteria) == 0:
                print("all dead")
                break
            print("***results after gen***", g)
            for b in self.bacteria:
                print(b.recombination)
                print(b.breed)
                print(b.energy)
                print(b.loc)

In [5]:
# testing of current simulation code
sim = Sim(10, 3, 2, 1, 0.5, 0.5, 0.5, 0.5, 1, 2, 5, 10)
print("***before***")
for b in sim.bacteria:
    print("*bac*")
    print("rec", b.recombination)
    print("breed", b.breed)
    print("energy", b.energy)
    print("loc", b.loc)
sim.simulate(10)
print("***after***")
for b in sim.bacteria:
    print("*bac*")
    print("rec", b.recombination)
    print("breed", b.breed)
    print("energy", b.energy)
    print("loc", b.loc)

***before***
*bac*
rec False
breed coop
energy 10
loc [3, 3]
*bac*
rec True
breed cheat
energy 10
loc [1, 3]
*bac*
rec True
breed coop
energy 10
loc [2, 4]
starting simulation
starting gen 0
***results after gen*** 0
False
coop
9.333333333333334
[3, 3]
True
cheat
10.333333333333334
[1, 3]
True
coop
9.333333333333334
[2, 4]
starting gen 1
***results after gen*** 1
False
coop
9.666666666666668
[4, 3]
True
cheat
10.666666666666668
[1, 2]
True
coop
8.666666666666668
[2, 5]
starting gen 2
***results after gen*** 2
False
coop
9.000000000000002
[4, 3]
True
cheat
11.000000000000002
[1, 2]
True
coop
8.000000000000002
[2, 5]
starting gen 3
***results after gen*** 3
False
coop
8.333333333333336
[4, 2]
True
cheat
10.333333333333336
[0, 2]
True
coop
6.333333333333336
[3, 5]
starting gen 4
***results after gen*** 4
False
coop
8.66666666666667
[4, 2]
True
cheat
9.66666666666667
[0, 2]
True
coop
5.666666666666669
[3, 5]
starting gen 5
***results after gen*** 5
False
coop
8.000000000000004
[3, 2]
True


# Replicating results from Lee et al. (2023)

## Compiled runs CSV

## Simulation graphics

In [6]:
# TODO: generate simulations accross a variety of params
# TODO: plot data + show simulation graphics for each independent simulation
# TODO: save end data in dictionary of simulation perfomances for future graphics

In [7]:
# TODO: create graphics for findings accross all simulations using data from dictionary

# Modifying the ABM

In [ ]:
# TODO: possibly add another factor to the simulation or another proccess to take place in each gen

# Simulating *in vitro* data

In [ ]:
# TODO: generate simulations that match in vivo data
# TODO: plot data + show simulation graphics for each independent simulation